#### Import libraries

In [1]:

import sys
sys.path.append('BEHRT/Early_integration/')
sys.path.append('BEHRT/')

from argparse import ArgumentParser
from Utils.common import *
from Utils.pytorch import load_model
from Utils.utils import age_vocab, input_vocab, mod_vocab, delay_vocab
from Utils.optimiser import adam
import random
import pytorch_pretrained_bert as Bert
from torch.utils.data import Dataset, DataLoader, Sampler
import pandas as pd
import sklearn.metrics as skm
import numpy as np
import torch
import torch.nn as nn
import os
import time
import warnings
warnings.filterwarnings(action='ignore')
from torch.utils.data.dataset import Dataset
from Utils.dataLoader_utils import seq_padding, random_mask, code2index,label2index, position_idx, index_seg
from Utils.add_endpoints import add_endp
from Utils.handle_file import handle_file
from Utils.FineTuneLoader import FinetuneDataset
from Utils.utils_for_classification import *
from torch.nn.parallel import DistributedDataParallel as DDP
hf = handle_file()
from Utils.preprocessing import cal_age, cal_therapies, spark_init, cal_subtherapies, int_to_list, flatten_list, int_to_str, cal_therapies_fr, divide_frame
from datetime import datetime, timedelta
from torch.utils.data import Dataset, DataLoader, Sampler
import sklearn
from sklearn.model_selection import StratifiedKFold
import math
import datetime 
from sklearn.metrics import roc_auc_score
from Utils import optimiser
from Utils.dataLoader_utils import ImbSampler, OverSampler
from Models.BertForClassification import BertForClassification
import warnings
warnings.filterwarnings(action='ignore')
from collections import Counter
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


2023-03-28 04:47:15.593041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 04:47:16.746643: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-28 04:47:16.746762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-28 04:47:16.746774: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

#### 1. Training model 

In [ ]:
def load_model(path, model):
    """Load pretrained weigths to  the model"""
    # load pretrained model and update weights
    pretrained_dict = torch.load(path).state_dict()
    model_dict = model.state_dict()
    
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model
    

In [ ]:
training_stats = list()        
def training(model, optim, config, e, train_dataloader, valid_dataloader, global_params, mlb):
    
    start = time.time()
    
    # Reset the loss at each epoch
    temp_loss, temp_f1 = [], []
    train_f1, train_loss = [], []
    train_targets, train_masks = [], []
    
    
    ## Count the number of batches
    cnt = 0
    count = 0
    # Training
    model.train()
    

    for step, batch in enumerate(train_dataloader):
         
        cnt +=1
        if config.age_in_inputs:
            input_ids, mod_ids, NPI_ids, del_ids, posi_ids, segment_ids, attMask, targets, _ = batch
            age_ids = None
            del_ids = del_ids.to(global_params['device'])
            
        if config.delays_in_inputs:
            input_ids, mod_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
            age_ids = None
            del_ids = None
        else:
            age_ids, input_ids, mod_ids, NPI_ids, del_ids, posi_ids, segment_ids, attMask, targets, _ = batch
            age_ids = age_ids.to(global_params['device'])
            del_ids = del_ids.to(global_params['device'])
             
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)
        
        ## Load batch to gpu 
        input_ids = input_ids.to(global_params['device'])
        mod_ids = mod_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        NPI_ids = NPI_ids.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
        
        ## Compute output (loss, logits and attentions scores) 
        output = model(input_ids = input_ids,modalities_ids= mod_ids, age_ids=age_ids,delays_ids= del_ids, seg_ids=segment_ids, posi_ids=posi_ids, NPI_ids=NPI_ids, attention_mask=attMask, labels=targets, output_attentions = True)
        
        loss = output.loss
        logits  = output.logits
        attentions = output.attentions
        
        if config['gradient_accumulation_steps'] > 1:
            loss = loss/config['gradient_accumulation_steps']
        
    
        temp_loss.append(loss.item())
        train_loss.append(loss.item())
        
        auc, aps, f1, recall, precision, _, _ = scores(logits, targets, training=True)
   
        temp_f1.append(f1)
        
        # Progress update every 100 batches
        if step % 100 == 0:
            print("epoch: {}\t| Cnt: {}\t| Loss: {}\t| f1 score: {}".format(e, cnt, np.mean(temp_loss), np.mean(temp_f1)))
            temp_loss, temp_f1 = [], []
        
        if (step + 1) % config['gradient_accumulation_steps'] == 0:
            optim.zero_grad()
            loss.sum().backward()
            optim.step()
        
        train_f1.append(f1)
        count+=1
        train_targets.append(targets)
        train_masks.append(attMask)
        logits = logits.cpu()
        targets = targets.cpu()
        loss = loss.cpu()
        torch.cuda.empty_cache()

    train_hidden_states = output.hidden_states[1:]
 
    # Measure how long this epoch took.
    training_time = format_time(time.time() - start)
                   
    print("")
    print("Average training loss: {0:.2f}".format(np.mean(train_loss)))
    print("Average training score: {0:.2f}".format(np.mean(train_f1)))
    print("Training epoch took: {:}".format(training_time))
    
          
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")
      
    valid_f1, valid_auc, valid_aps, valid_recall, valid_precision, valid_loss, valid_time, valid_hidden_states, valid_masks = evaluating(model, config, valid_dataloader, global_params, mlb)
    
    # Record all statistics from this epoch.
    training_stats.append({'epoch': e+1, 
                               'Training Loss': np.mean(train_loss) ,
                               'Training Time' : training_time,
                               'Average Train f1 score' : np.mean(train_f1),
                               'Validation Loss': np.mean(valid_loss), 
                               'Validating Time' : valid_time, 
                               'Average Valid f1 score' : np.mean(valid_f1),
                               'Average Valid ROC-AUC score' : np.mean(valid_auc), 
                               'Average Valid precision' : np.mean(precision),
                               'Average Valid recall' : np.mean(recall),
                                })  
    
    
    return valid_f1, train_f1, valid_loss, train_loss, valid_auc,  valid_aps, training_stats, train_hidden_states, valid_hidden_states, train_targets, train_masks, valid_masks
    

In [ ]:
def evaluating(model, config, data, global_params, mlb):
                               
    start = time.time()
                               
    model.eval()
    y_pred, y_true = [], []
    loss, masks = [], []
    
    for step, batch in enumerate(data):
        
        if config.age_in_inputs:
            input_ids, mod_ids, del_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
            age_ids = None
            del_ids = del_ids.to(global_params['device'])
            
        if config.delays_in_inputs:

            input_ids, mod_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
            age_ids = None
            del_ids = None
        else:
            age_ids, input_ids, mod_ids,  NPI_ids, del_ids, posi_ids, segment_ids, attMask, targets, _ = batch
            age_ids = age_ids.to(global_params['device'])
            del_ids = del_ids.to(global_params['device'])
             
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)
    
        ## Load batch to gpu 
        input_ids = input_ids.to(global_params['device'])
        mod_ids = mod_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        NPI_ids = NPI_ids.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
      
        with torch.no_grad():
            outputs = model(input_ids=input_ids, modalities_ids= mod_ids, age_ids= age_ids, delays_ids=del_ids, seg_ids=segment_ids,posi_ids= posi_ids, NPI_ids=NPI_ids, attention_mask=attMask, labels=targets, output_attentions=True)
        
        #loss.append(outputs.loss.sum().item())
        loss.append(outputs.loss.item())
        
        
        logits = outputs.logits.cpu()
        targets = targets.cpu()
        
        y_true.append(targets)
        y_pred.append(logits)
        
        masks.append(attMask)
        torch.cuda.empty_cache()
        
    hidden_states = outputs.hidden_states[1:]
    
    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0) 
    
    auc, aps, f1, recall, precision, logits, labels = scores(y_pred, y_true)
   
    # Measure how long this epoch took.
    val_time = format_time(time.time() - start)
    
    
    print("Validation Loss: {0:.2f}".format(np.mean(loss)))
    print("Validation took: {:}".format(val_time))
    print()

    return f1, auc, aps, recall, precision, loss, val_time, hidden_states, masks


#### Hyperparameters tuning with Weights & Biases

In [ ]:
import wandb


os.environ['https_proxy'] = ""
os.environ['http_proxy'] = ""
os.environ['no_proxy']= ""
os.environ['HTTPS_PROXY']= ""
os.environ['HTTP_PROXY']= ""
os.environ['NO_PROXY']= ""

os.environ['WANDB_NOTEBOOK_NAME'] = 'Tabular_BEHRT/Tasks/Binary_classification.ipynb'
os.environ['WANDB_API_KEY'] = ""
os.environ['WANDB_NAME'] = "Binary_classification"
os.environ['WANDB_NOTES'] = "Hyperparameters tuning using wandb for binary clf: Bayesian and hyperband scheduler"
os.environ['WANDB_AGENT_MAX_INITIAL_FAILURES']='0'
os.environ['WANDB__SERVICE_WAIT'] ="300"
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"


wandb.login()

#### Run Binary classification 
1. Config

In [6]:
pretrain_model_path =  ""   # pretrained MLM path

file_config = { 'data':"",
                'labels' : "" , 
                'test_idxs' : "",

                }

global_params = {
    'device': device,
    'output_dir': "", # output folder
    'best_name': 'best_clf',   # output model name
    'max_len_seq': 512, #100,
    'max_age': 110,
    'max_delay': 30, ## = 30years
    'age_month': 12,
    'delay_month': 0.25,
    'age_year': False,
    'age_symbol': None,
    'min_visit': 5,
    
}

train_params = {
    'use_cuda': True,
    'max_len_seq': global_params['max_len_seq'],
    'device': device,
}

feature_dict = {
    'word':True,
    'seg':True,
    'age':False,
    'modalities': True,
    'delays': False,
    'position': True, 
    'NPI' : True
}


2. Import data 

In [ ]:
"""Import train data """
data =  hf._load_pkl(file_config['data'])

"""Import labels """
labels = hf._load_pkl(file_config['labels'])

"""Merge label and data """
data = pd.merge(data, labels, on=['Num_dossier'])

# Display 3 first rows
data.head(1)

In [8]:
index_censored = data.loc[((data['delay_dfs_surg'] < 35.9951) & (data['label'] == 0))]['Num_dossier'].values.tolist()
index_relapse_1y = data.loc[((data['status_dfs'] == 1) & (data['delay_dfs_surg'] < 11.9983))]['Num_dossier'].values.tolist()

data = data.loc[~data['Num_dossier'].isin(index_censored_3y+index_relapse_1y)]#['status_dfs_surg_3y'].value_counts()#.sum()  ## 8124
print(data.shape, len(index_censored_3y), len(index_relapse_1y))

10029

In [11]:
# remove patients with visits less than min visit
previous_shape = data.shape[0]

data['length'] = data['inputs_normal_range_and_delta_age_and_delays_preprocessed'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
data = data[data['length'] >= global_params['min_visit']]
#data = data.reset_index(drop=True)

print("We lost {} patient".format(previous_shape - data.shape[0]))

We lost 0 patient


In [13]:
## use the vocabulary from the entire history dataset

whole_seq = hf._load_pkl('')
tokenVocab, _ = input_vocab(inputs = whole_seq.inputs, symbol=global_params['age_symbol'])
modalitiesVocab, _ = mod_vocab(whole_seq.modalities, symbol=global_params['age_symbol'])
NPIVocab = {'PAD':0, 'UNK':1, 'VPPG':2, 'PPG':3, 'MPGI':4, 'MPGII':5, 'GPG':6, 'EPG':7}

# Binary classification 
labelVocab = {0:0, 1:1}

del whole_seq

4. Prepare data 

In [ ]:
test_idxs = hf._load_pkl(file_config['test_idxs'])

test = data.loc[data['Num_dossier'].isin(test_idxs.values.tolist())]
train =  data.loc[~data['Num_dossier'].isin(test_idxs.values.tolist())] 

print(train.shape, test.shape)

In [19]:
train['patid'] = range(len(train))
test['patid'] = range(len(test))

train.index =  range(len(train))
test.index =  range(len(test))

3. Config and utils

In [1]:
## Config for the task with sweep (wandb)

config = {'method' : 'random', 
                'metric' : {'name' : 'valid_aps', 'goal': 'maximize'},
                'parameters' : {'hidden_size': {'value': 288},       # word embedding and seg embedding hidden size
                'intermediate_size': {'value': 210},                # the size of the "intermediate" layer in the transformer encoder
                'hidden_dropout_prob': {'values': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},                    # dropout rate
                'num_hidden_layers': {'value': 5},                   # number of multi-head attention layers required
                'num_attention_heads': {'value': 6},                  # number of attention heads
                'attention_probs_dropout_prob': {'values': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
                'hidden_act': {'value' :'gelu'},                                                 # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
                'vocab_size': {'value': len(tokenVocab)},                                                  # number of disease + symbols for word embedding
                'seg_vocab_size': {'value':2},                                                             # number of vocab for seg embedding
                'modalities_vocab_size': {'value':len(modalitiesVocab)}, 
                'age_vocab_size': {'value':1},                                                 # number of vocab for age embedding
                'delays_vocab_size': {'value':1},
                'NPI_vocab_size' : {'value': len(NPIVocab)},
                'epochs': {'values': [15]},
                'max_position_embeddings': {'value': train_params['max_len_seq']},                          # maximum number of tokens
                'initializer_range': {'value' : 0.02},                                                    # parameter weight initializer range
                'lr' : {'values': [5e-6, 1e-5, 3e-5, 5e-5, 1e-4, 3e-4]},                      
                'warmup_proportion': {'value': 0.1}, 
                'weight_decay' : {'values': [0, 0.0001]},#0.0005, 0.001, 0.005, 0.01, 0.005]}, 
                'class_weights': {'values': [10, 20, 40, 60]},
                'batch_size': {'value': 24},   # integers between 8 and 32 # with evenly-distributed logarithms
                'gradient_accumulation_steps':{'distribution': 'int_uniform', 'min':4, 'max':10},
                'chunk_size_feed_forward' : {'value':0},
                'output_attentions' : {'value':True},
                'is_decoder' : {'value':False},
                'add_cross_attention':{'value': False},
                'layer_norm_eps' : {'value': 1e-5},
                'delays_in_inputs': {'value': True},
                'activation': {'value':False},
                'age_in_inputs': {'value': False}, 
                'freeze_weights': {'value': False},},     
              #  'early_terminate': {'type': 'hyperband', 's': 2, 'eta': 3, 'max_iter': 27},
}   

from transformers.configuration_utils import PretrainedConfig

class BertConfig(PretrainedConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embeddings'),
            position_embedding_type = config.get('position_embedding_type'),
            initializer_range=config.get('initializer_range'),
            
        )
        self.delays_vocab_size = config.get('delays_vocab_size')
        self.modalities_vocab_size = config.get('modalities_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.NPI_vocab_size = config.get('NPI_vocab_size')
        self.output_attentions = config.get('output_attentions')
        self.chunk_size_feed_forward = config.get('chunk_size_feed_forward')
        self.is_decoder = config.get('is_decoder')
        self.layer_norm_eps = config.get('layer_norm_eps')
        self.add_cross_attention = config.get('add_cross_attention')
        self.activation = config.get('activation')
        self.age_in_inputs = config.get('age_in_inputs')
        self.delays_in_inputs = config.get('delays_in_inputs')
        self.freeze_weights = config.get('freeze_weights')
        self.class_weights = config.get('class_weights')
        self.freeze_weights = config.get('freeze_weights')
        self.gradient_accumulation_steps = config.get('gradient_accumulation_steps')
        
  

NameError: name 'tokenVocab' is not defined

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(labelVocab.values()))
mlb.fit([[each] for each in list(labelVocab.values())])

MultiLabelBinarizer(classes=[0, 1])

In [ ]:
""" Check if the gpu is available """
if torch.cuda.is_available():
   
    device = torch.device("cuda")
    n_gpus = torch.cuda.device_count()
    print("There are %d GPU(s) available. " %torch.cuda.device_count())
    print("We will use the GPU: {}".format(torch.cuda.get_device_name(0)))
    
else:
    print("No GPU available, using the CPU instead")
    device = torch.device("cpu")

4. Sweep settings for hp tuning in W&B

In [ ]:
sweep_id = wandb.sweep(config)

In [33]:
## Train the model 

def define_model(config, pretraining = True):
   
    conf = BertConfig(config)
    model = BertForClassification(conf, num_labels=conf.num_labels, feature_dict=feature_dict, class_weights = conf.class_weights)
    if pretraining:
        model = load_model(pretrain_model_path, model)   
    
  #  model = nn.DataParallel(model).cuda()
    model = model.cuda()
    optim = optimiser.adam(params=list(model.named_parameters()), config=config)
 
    return model, optim

def load_data(data, config, sampling=True, X_tr=None, y_tr=None):
    """
    Create a torch DataLoader that can be used to load the data batch by batch.
    The batch are balanced using NewBatchOverSampler.
    """  
    Dset = FinetuneDataset(token2idx = tokenVocab, label2idx=labelVocab, mod2idx=modalitiesVocab, 
                   NPI2idx=NPIVocab, dataframe=data, max_len=global_params['max_len_seq'], 
                     code='inputs_normal_range_and_delta_age_and_delays_preprocessed',
                     mod='modalities_for_twos_and_age_and_delays_preprocessed', NPI='NPI_for_twos_age_delays', age_in_inputs=config.age_in_inputs, delays_in_inputs=config.delays_in_inputs)

    if sampling:
        return DataLoader(dataset=Dset, batch_sampler=StratifiedSampler(X_tr, y_tr, batch_size=config['batch_size']))
    else:
     #   return DataLoader(dataset=Dset, batch_size=config.batch_size,  sampler = torch.utils.data.RandomSampler(data.index))
        return DataLoader(dataset=Dset, batch_size=config['batch_size'], shuffle=False)

    

In [ ]:

def train_per_epochs(model, optim, config, trainload, validload, testload, mlb):
  #  empty_cuda()  
    train_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch =  [] , [], []
    valid_hidden_states_per_epoch, valid_targets_per_epoch, valid_mask_per_epoch = [], [], []  
    valid_f1_list, train_f1_list, valid_loss_list, train_loss_list,  valid_auc_list , valid_aps_list= [], [], [], [], [], []
    best_score = 0.0
    best_temp_auc = 0.7
    
    for e in range(config['epochs']):
        
        valid_f1, train_f1, valid_loss, train_loss, valid_auc, valid_aps, training_stats, train_hidden_states, valid_hidden_states, train_targets, train_masks, valid_masks =  training(model, optim, config, e, trainload, validload, global_params, mlb)
        train_hidden_states_per_epoch.append(train_hidden_states)
        valid_hidden_states_per_epoch.append(valid_hidden_states)
        train_targets_per_epoch.append(train_targets)
        train_mask_per_epoch.append(train_masks)
        valid_mask_per_epoch.append(valid_masks)

        train_f1_list.append(np.mean(train_f1))
        train_loss_list.append(np.mean(train_loss))
        valid_f1_list.append(np.mean(valid_f1))
        valid_auc_list.append(np.mean(valid_auc))
        valid_loss_list.append(np.mean(valid_loss))
        valid_aps_list.append(np.mean(valid_aps))

        mean_aps = np.mean(valid_aps)
        
        if mean_aps > best_score: 
            model_to_save = model.module if hasattr(model, 'module') else model 
            output_model_file = os.path.join(global_params['output_dir'], global_params['best_name'])
            create_folder(global_params['output_dir'])
            
            torch.save(model_to_save.state_dict(), output_model_file)
            best_score = mean_aps
            
        print('valid score: {}'.format(np.mean(valid_aps)))
        print()
        
        wandb.log({'f1': np.mean(train_f1), 'loss': np.mean(train_loss), 'valid_f1': valid_f1, 'valid_auc':valid_auc,
                      'valid_aps':valid_aps, 'valid_loss':np.mean(valid_loss), 'best_aps':best_score})
            

    test_f1, test_auc, test_aps, test_recall, test_precision, test_loss, test_time, test_hidden_states, test_masks = evaluating(model_to_save, config, testload, global_params, mlb)
        
    wandb.log({'test_f1':test_f1, 'test_auc': test_auc, 'test_aps': test_aps, 'test_recall': test_recall, 'test_precision':test_precision})

    if test_auc >= best_score:
        wandb.alert(
        title="Higher auc", 
        text=f"3years: You reach your higher roc: {test_auc}",
        )
        torch.save(model_to_save.state_dict(), f'Tabular_BEHRT/Outs/clf_model_{test_auc}')

    return model_to_save, test_f1, test_auc, test_aps, test_recall, test_precision, test_loss, train_hidden_states_per_epoch, valid_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch, valid_mask_per_epoch, valid_f1_list, train_f1_list, valid_loss_list, train_loss_list,  valid_auc_list , valid_aps_list
   
    
def train_model(config=None, train=train, test=test, mlb=mlb, inputs ='inputs_normal_range_and_delta_age_and_delays_preprocessed', class_weights = False, cross_val = False, n_splits=5, seed=2):
    empty_cuda()
    with wandb.init(config = config) as run: 
        config = wandb.config
   
        set_seed(seed)
        test.index = range(len(test))
        test.patid = range(len(test))
        test_loader = load_data(test, config, sampling=False)

        if cross_val: 

            tot_train_f1, tot_train_loss, tot_val_f1, tot_val_loss, tot_val_auc, tot_val_aps, tot_test_f1, tot_test_loss, tot_test_auc, tot_test_aps, tot_test_recall, tot_test_precision = [], [], [], [], [], [], [], [], [], [], [], []
            n_splits = n_splits
            skf = StratifiedKFold(n_splits = n_splits)


            for i, (train_idx, valid_idx) in enumerate(skf.split(train.drop(['label'], axis=1), train['label'])):
                train_df = train.iloc[train_idx]
                cv_df = train.iloc[valid_idx]

                train_df.index = range(train_df.shape[0])
                train_df.patid = range(train_df.shape[0])
                X_tr = train_df[inputs]
                y_tr = train_df[inputs]

                cv_df.index = range(cv_df.shape[0])
                cv_df.patid = range(cv_df.shape[0])

                if class_weigths:
                    train_loader = load_data(train_df, config, sampling = False)
                else:
                    train_loader = load_data(train_df, config, X_tr=X_tr, y_tr=y_tr)

                valid_loader = load_data(cv_df, config, sampling = False)

                model, optim= define_model(config, pretraining = False)

                if config['freeze_weights']:
                   # param_to_freeze = ['bert.embeddings.word_embeddings.weight','bert.embeddings.segment_embeddings.weight','bert.embeddings.modalities_embeddings.weight','bert.embeddings.age_embeddings.weight', 'bert.embeddings.delays_embeddings.weight', 'bert.embeddings.NPI_embeddings.weight', 'bert.embeddings.posi_embeddings.weight']
                    for name, param in model.named_parameters():
                       # if name in param_to_freeze:
                        param.requires_grad = False

                trained_model, test_f1, test_auc, test_aps,  test_recall, test_precision, test_loss, train_hidden_states_per_epoch, valid_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch, valid_mask_per_epoch,  valid_f1_list, train_f1_list, valid_loss_list, train_loss_list, valid_auc_list , valid_aps_list = train_per_epochs(model, otpim, config, train_loader, valid_loader, test_loader, mlb)


            tot_train_f1.append(np.mean(train_f1_list))
            tot_train_loss.append(np.mean(train_loss_list))
            tot_val_f1.append(np.mean(valid_f1_list))
            tot_val_auc.append(np.mean(valid_auc_list))
            tot_val_aps.append(np.mean(valid_aps_list))
            tot_test_f1.append(test_f1)
            tot_test_auc.append(test_auc)
            tot_test_aps.append(test_aps)
            tot_test_recall.append(test_recall)
            tot_test_loss.appen(test_loss)

            return trained_model, tot_train_f1, tot_train_loss, tot_val_f1, tot_val_loss, tot_val_auc, tot_val_aps, tot_test_f1, tot_test_loss, tot_test_auc, tot_test_aps, tot_test_recall, tot_test_precision, train_hidden_states_per_epoch, valid_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch, valid_mask_per_epoch, 

        else: 


            X_train, X_valid, y_train, y_valid = train_test_split(train.drop(['label'], axis=1), train['label'], test_size=0.2, random_state=42, stratify=train['label'])

            train_df = pd.concat([X_train, y_train], axis=1)
            cv_df = pd.concat([X_valid, y_valid], axis=1)

            train_df.index = range(train_df.shape[0])
            train_df.patid = range(train_df.shape[0])

            X_tr = train_df[inputs]
            y_tr = train_df['label']

            cv_df.index = range(cv_df.shape[0])
            cv_df.patid = range(cv_df.shape[0])

            if class_weights:
                train_loader = load_data(train_df, config, sampling = False)
            else:
                train_loader = load_data(train_df, config, X_tr=X_tr, y_tr=y_tr)

            valid_loader = load_data(cv_df, config, sampling=False)
            model, optim = define_model(config, pretraining=False)

            if config['freeze_weights']:
                for param in model.bert.parameters():
                    param.requires_grad = False

            trained_model, test_f1, test_auc, test_aps, test_recall, test_precision, test_loss, train_hidden_states_per_epoch, valid_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch, valid_mask_per_epoch,  valid_f1_list, train_f1_list, valid_loss_list, train_loss_list, valid_auc_list, valid_aps_list = train_per_epochs(model, optim, config, train_loader, valid_loader, test_loader, mlb)
        
        return trained_model, np.mean(train_f1_list), np.mean(train_loss_list), np.mean(valid_f1_list), np.mean(valid_loss_list), np.mean(valid_auc_list), np.mean(valid_aps_list), test_f1, test_loss, test_auc, test_aps, test_recall, test_precision, train_hidden_states_per_epoch, valid_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch, valid_mask_per_epoch, 



### 5. Training

In [ ]:

if __name__ == "__main__":
       
    # ========================================
    #               Training
    # ========================================
    
    start = time.time()

    ## lauch W&B tuning with 15 runs
    
    wandb.agent(sweep_id = sweep_id, function=train_model, count=15)
  
    trained_model, train_f1, train_loss, valid_f1, valid_loss, valid_auc , valid_aps, test_f1, test_loss, test_auc, test_aps, test_recall, test_precision, train_hidden_states_per_epoch, valid_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch, valid_mask_per_epoch = train_model(config = config)
    print("")
    print("Training complete!")
    print("Total training took {:} (h:mm:ss)".format(format_time(end-start)))
